In [16]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
data_train = pd.read_csv("/kaggle/input/amazon-reviews/train.csv",header=None, names=("polarity", "title", "text"))
data_train = data_train.sample(1000000, random_state=99)
data_test = pd.read_csv("/kaggle/input/amazon-reviews/test.csv",header=None, names=("polarity", "title", "text"))

In [18]:
data_train['text'] = data_train['title'] + data_train['text']
data_test['text'] = data_test['title'] + data_test['text']

In [19]:
data_train

,polarity,title,text
3277024,1,does this man know anything?,does this man know anything?I frown at such ig...
2620776,2,If Only Amazon allowed my 6 *s.,"If Only Amazon allowed my 6 *s.Wow,This movie ..."
1588872,2,A must have item for anyone on the go,A must have item for anyone on the goA lot of ...
3125695,2,Very Helpful,Very HelpfulI bought this book after joining C...
1668957,1,Academically Repugnant,Academically RepugnantThe first ten minutes of...
...,...,...,...
1787985,1,Not really a release by the Cuban timba band B...,Not really a release by the Cuban timba band B...
1546483,1,Do not recomment this Bird-X TX-PRO transonic ...,Do not recomment this Bird-X TX-PRO transonic ...
158073,2,The world of the bookseller,The world of the booksellerGood story about th...
3337847,2,"clean, light, scent","clean, light, scentI've been wearing this perf..."


In [20]:
data_train = data_train.drop('title', axis=1)
data_test = data_test.drop('title', axis=1)


In [21]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [22]:
data_train['polarity'] = data_train['polarity'] - 1
data_test['polarity'] = data_test['polarity'] - 1

In [23]:
len(data_train), len(data_test)

(1000000, 400000)

In [24]:
data_train = data_train.dropna()
data_train = data_train.drop_duplicates()
data_test = data_test.dropna()
data_test = data_test.drop_duplicates()


In [25]:
X_train = [clean_text(text) for text in data_train['text']]
X_test = [clean_text(text) for text in data_test['text']]
y_train = data_train['polarity'].values
y_test = data_test['polarity'].values

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')


2025-06-08 11:43:21.789648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749383002.077598      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749383002.160492      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [27]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

embedding_dim = 64
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    SimpleRNN(64, return_sequences=False),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


2025-06-08 11:46:43.748371: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [28]:
model.fit(X_train_pad, y_train, epochs=5, batch_size=128, validation_split=0.2)

Epoch 1/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 247s 38ms/step - accuracy: 0.7735 - loss: 0.4325 - val_accuracy: 0.8949 - val_loss: 0.2522
Epoch 2/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 238s 38ms/step - accuracy: 0.9009 - loss: 0.2445 - val_accuracy: 0.8967 - val_loss: 0.2571
Epoch 3/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 236s 38ms/step - accuracy: 0.8908 - loss: 0.2658 - val_accuracy: 0.9094 - val_loss: 0.2254
Epoch 4/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 245s 39ms/step - accuracy: 0.9168 - loss: 0.2111 - val_accuracy: 0.9036 - val_loss: 0.2408
Epoch 5/5
6250/6250 ━━━━━━━━━━━━━━━━━━━━ 242s 39ms/step - accuracy: 0.9234 - loss: 0.1967 - val_accuracy: 0.8966 - val_loss: 0.2514


In [29]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 100, 64)             │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,951,109 (7.44 MB)

 Trainable params: 650,369 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,300,740 (4.96 MB)

In [30]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


12500/12500 ━━━━━━━━━━━━━━━━━━━━ 117s 9ms/step - accuracy: 0.8951 - loss: 0.2564
Test Loss: 0.2510
Test Accuracy: 0.8972


In [32]:
y_pred_probs = model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype("int32")  

12500/12500 ━━━━━━━━━━━━━━━━━━━━ 110s 9ms/step


In [33]:
from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix(y_test, y_pred)

array([[191729,   8255],
       [ 32865, 167127]])

In [34]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.85      0.96      0.90    199984\n           1       0.95      0.84      0.89    199992\n\n    accuracy                           0.90    399976\n   macro avg       0.90      0.90      0.90    399976\nweighted avg       0.90      0.90      0.90    399976\n'